In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import xgboost

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler


from keras.models import Sequential
from keras.layers import Dense, LSTM, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.regularizers import l2
from time import time

# prepare data for lstm
from pandas import read_csv
from pandas import DataFrame
from pandas import concat

C:\Users\kanan\anaconda3\envs\weatherforecast\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
#Reading the CSV file
df = pd.read_csv("C:\\Users\\kanan\\Desktop\\Study Material\\In-house CP\\Untitled Folder\\cleaned_weatherAUS.csv")

Direction = ['N','NNE','NE','ENE','E','ESE','SE','SSE','S','SSW','SW','WSW','W','WNW','NW','NNW']
Degree = []

x = 0
for i in range(0,337,22):
    Degree.append(i+x)
    x = x + 0.5
    
Direction_Degree = dict(zip(Direction,Degree))
print(Direction_Degree)

#Converting WindGustDir, WindDir9am, WindDir3pm to degree format

df = df.replace({"WindGustDir": Direction_Degree})
df = df.replace({"WindDir9am": Direction_Degree})
df = df.replace({"WindDir3pm": Direction_Degree})

df = df.drop(columns = ['Date','Location','RainTomorrow'],axis=1)

from sklearn.preprocessing import LabelEncoder

cat_f = [x for x in df.columns if df[x].dtype == 'object']
print(cat_f)

# traversing all columns for converting into continuous valriables
for name in cat_f:
    enc = LabelEncoder()
    enc.fit(list(df[name].values.astype('str')) + list(df[name].values.astype('str')))
    df[name] = enc.transform(df[name].values.astype('str'))

{'N': 0, 'NNE': 22.5, 'NE': 45.0, 'ENE': 67.5, 'E': 90.0, 'ESE': 112.5, 'SE': 135.0, 'SSE': 157.5, 'S': 180.0, 'SSW': 202.5, 'SW': 225.0, 'WSW': 247.5, 'W': 270.0, 'WNW': 292.5, 'NW': 315.0, 'NNW': 337.5}
['RainToday']


In [3]:
# Dividing Data into features and target for classifier
X = df.drop(['Rainfall'], axis=1)
Y = df['Rainfall']

In [4]:
# our test dataset doesn't have a target variable, so we'll have to test on the train df using holdout 
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [5]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [6]:
model = xgboost.XGBRegressor(objective ='reg:linear',
                  n_estimators = 1000, seed = 0).fit(x_train, y_train)

[15:39:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.


In [7]:
results = model.predict(x_test)

In [8]:
print(mean_squared_error(y_test, results))
print(mean_absolute_error(y_test, results))
print(r2_score(y_test,results))

0.04066601976472775
0.12147781713032867
0.7372001294403192
